In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver 
import json
import time
from tqdm.auto import tqdm



In [2]:
# using webdriver to extract page information:
cService = webdriver.ChromeService(executable_path='../../web_scrape_projects/chromedriver.exe')
driver = webdriver.Chrome(service = cService)


In [3]:
tickers = ["AEE","REZ","1AE","1MC","NRZ"]

In [8]:
# instantiating an empty dataframe with the expected columns:

output_df = pd.DataFrame(columns= ['id', 'document_release_date', 'document_date', 'url', 'relative_url',
       'header', 'market_sensitive', 'number_of_pages', 'size',
       'legacy_announcement', 'issuer_code', 'issuer_short_name',
       'issuer_full_name'])

In [9]:
def get_ticker_info(ticker):
    
    base_url = f"https://www.asx.com.au/asx/1/company/{ticker}/announcements?count=20&market_sensitive=false"
    driver.get(base_url)
    res = driver.page_source

    info = BeautifulSoup(res,'html.parser') 
    data = json.loads(info.find('pre').get_text())

    output = pd.DataFrame(data["data"])

    return output

In [10]:
for ticker_ in tqdm(tickers):

    df = get_ticker_info(ticker_)
    output_df = pd.concat([output_df, df], ignore_index=True)

    time.sleep(5)

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
output_df['issuer_code'].value_counts()

issuer_code
AEE    20
REZ    20
1AE    20
1MC    20
NRZ    20
Name: count, dtype: int64

In [12]:
output_df.to_csv('./data/test_data.csv', index=False)

## Displaying Data Frame Output:

In [13]:
output_df.head(2)

,id,document_release_date,document_date,url,relative_url,header,market_sensitive,number_of_pages,size,legacy_announcement,issuer_code,issuer_short_name,issuer_full_name
0,02839968,2024-08-15T14:53:40+1000,2024-08-15T14:52:28+1000,https://announcements.asx.com.au/asxpdf/202408...,/asxpdf/20240815/pdf/066nj3f5lb1dpm.pdf,Notification regarding unquoted securities - AEE,False,7,21.5KB,False,AEE,AURA EN,AURA ENERGY LIMITED
1,02839899,2024-08-15T11:11:09+1000,2024-08-15T11:10:29+1000,https://announcements.asx.com.au/asxpdf/202408...,/asxpdf/20240815/pdf/066nb5j4zm2h0r.pdf,Proposed issue of securities - AEE,False,5,17.8KB,False,AEE,AURA EN,AURA ENERGY LIMITED


In [14]:
# identifying Trading halt announcements:

In [16]:
for idx, header in enumerate(output_df['header']):
    if "Trading Halt" in str(header):
        print(f" --- found : {output_df.loc[idx, ['header','issuer_code']]}  ---")

 --- found : header         Trading Halt
issuer_code             REZ
Name: 22, dtype: object  ---
 --- found : header         Trading Halt
issuer_code             NRZ
Name: 90, dtype: object  ---


**for the purpose of this assessment, table display for the 'Trading Halt' filter would be limited to columns :**
- 'issuer_code'
- 'issuer_full_name'